In [1]:
# Standard libraries
import xarray as xr
import numpy as np
import pandas as pd
# import os
# from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import iris
from iris.pandas import as_cubes
import sys

from datetime import datetime
from cartopy.util import add_cyclic_point
import gc
import imageio.v2
from IPython import display
# import netCDF4
from global_land_mask import globe
# # Import tobac itself:
import tobac

# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

## _Sorting out cells that made it to landfall (used the globe.is_land() library)_

In [2]:
track = pd.read_csv('../saved-files/threshold-20/final-tracks-threshold-20.csv')
track

frame  idx      hdim_1      hdim_2  num  threshold_value  feature  \
0          0    3  138.459946  209.639577   12               20        3   
1          1    2  142.106890  211.038006   11               20       56   
2          2    3  144.716901  212.259109    4               20      107   
3          3    3  148.635454  213.044500    3               20      148   
4          0    7  179.772123  360.749762    7               20        7   
...      ...  ...         ...         ...  ...              ...      ...   
12803    299    1  132.210510  264.802250    9               20    15497   
12804    300    1  134.016782  265.607027    8               20    15522   
12805    298   15  209.325817  289.439165    8               20    15481   
12806    299   12  210.187722  291.164259    7               20    15508   
12807    300   12  212.000000  292.056669    3               20    15533   

                      time              timestr  projection_y_coordinate  \
0      2020-03-13 04:00:00  2020-03-13 04:00:00            -2.198460e+06   
1      2020-03-13 04:05:00  2020-03-13 04:05:00            -2.202107e+06   
2      2020-03-13 04:10:00  2020-03-13 04:10:00            -2.204717e+06   
3      2020-03-13 04:15:00  2020-03-13 04:15:00            -2.208635e+06   
4      2020-03-13 04:00:00  2020-03-13 04:00:00            -2.239772e+06   
...                    ...                  ...                      ...   
12803  2020-03-14 04:55:00  2020-03-14 04:55:00            -2.192211e+06   
12804  2020-03-14 05:00:00  2020-03-14 05:00:00            -2.194017e+06   
12805  2020-03-14 04:50:00  2020-03-14 04:50:00            -2.269326e+06   
12806  2020-03-14 04:55:00  2020-03-14 04:55:00            -2.270188e+06   
12807  2020-03-14 05:00:00  2020-03-14 05:00:00            -2.272000e+06   

       projection_x_coordinate   latitude  longitude  cell        time_cell  \
0                126639.576617  70.178065  13.296810     3  0 days 00:00:00   
1                128038.005949  70.144203  13.327626     3  0 days 00:05:00   
2                129259.108875  70.119843  13.355321     3  0 days 00:10:00   
3                130044.500153  70.083824  13.369686     3  0 days 00:15:00   
4                277749.761570  69.679007  17.069049     7  0 days 00:00:00   
...                        ...        ...        ...   ...              ...   
12803            181802.250223  70.199616  14.740749  3835  0 days 00:05:00   
12804            182607.027340  70.182629  14.757736  3835  0 days 00:10:00   
12805            206439.164789  69.480737  15.197855  3837  0 days 00:00:00   
12806            208164.259129  69.471488  15.239071  3837  0 days 00:05:00   
12807            209056.668584  69.454306  15.257232  3837  0 days 00:10:00   

         distance   mean dBZ    max dBZ  area($km^2$)  
0      147.586718  13.245867  34.567482     28.850011  
1      144.153937  12.974534  35.577388     23.876203  
2      141.544931  12.780750  34.063904     16.911861  
3      138.460932  12.708913  36.584485     12.933215  
4       64.097510  13.245867  34.567482     13.928206  
...           ...        ...        ...           ...  
12803  117.272271  15.240493  41.116443     11.937888  
12804  115.288355  14.702386  40.106802      9.947996  
12805   41.339728  15.301029  39.603226     20.893423  
12806   39.445606  15.240493  41.116443     16.913434  
12807   37.683037  14.702386  40.106802     14.923758  

[12808 rows x 19 columns]

In [3]:
track.head()   

frame  idx      hdim_1      hdim_2  num  threshold_value  feature  \
0      0    3  138.459946  209.639577   12               20        3   
1      1    2  142.106890  211.038006   11               20       56   
2      2    3  144.716901  212.259109    4               20      107   
3      3    3  148.635454  213.044500    3               20      148   
4      0    7  179.772123  360.749762    7               20        7   

                  time              timestr  projection_y_coordinate  \
0  2020-03-13 04:00:00  2020-03-13 04:00:00            -2.198460e+06   
1  2020-03-13 04:05:00  2020-03-13 04:05:00            -2.202107e+06   
2  2020-03-13 04:10:00  2020-03-13 04:10:00            -2.204717e+06   
3  2020-03-13 04:15:00  2020-03-13 04:15:00            -2.208635e+06   
4  2020-03-13 04:00:00  2020-03-13 04:00:00            -2.239772e+06   

   projection_x_coordinate   latitude  longitude  cell        time_cell  \
0            126639.576617  70.178065  13.296810     3  0 days 00:00:00   
1            128038.005949  70.144203  13.327626     3  0 days 00:05:00   
2            129259.108875  70.119843  13.355321     3  0 days 00:10:00   
3            130044.500153  70.083824  13.369686     3  0 days 00:15:00   
4            277749.761570  69.679007  17.069049     7  0 days 00:00:00   

     distance   mean dBZ    max dBZ  area($km^2$)  
0  147.586718  13.245867  34.567482     28.850011  
1  144.153937  12.974534  35.577388     23.876203  
2  141.544931  12.780750  34.063904     16.911861  
3  138.460932  12.708913  36.584485     12.933215  
4   64.097510  13.245867  34.567482     13.928206

In [4]:
%%time
cell_landfall = []
for i, j in enumerate(track['cell']):
#     # # Check if a point is on land:
    lat = track['latitude'][i]
    lon = track['longitude'][i]
    is_on_land = globe.is_land(lat, lon)
    if is_on_land:
        # if ((lat >= 69.0) & (lon >= 12.0)):
            # print(f"lat={lat}, lon={lon} is on land: {is_on_land} -- cell_id={j}")
        cell_landfall.append(j)

CPU times: user 1.33 s, sys: 1.01 ms, total: 1.33 s
Wall time: 4.14 s


In [5]:
%%time
cells_landfall=np.unique(cell_landfall)
cells_landfall

CPU times: user 205 µs, sys: 0 ns, total: 205 µs
Wall time: 208 µs


array([   8,   20,   32,   92,  101,  108,  110,  115,  117,  127,  131,
        136,  142,  165,  177,  201,  211,  231,  232,  236,  254,  260,
        268,  281,  283,  297,  300,  322,  341,  370,  371,  377,  384,
        388,  402,  405,  411,  427,  428,  441,  449,  451,  482,  525,
        546,  579,  585,  634,  668,  670,  676,  726,  730,  746,  829,
        862,  870,  871,  885,  886,  889,  898,  908,  931,  937,  977,
        998, 1000, 1010, 1016, 1026, 1041, 1055, 1075, 1082, 1090, 1108,
       1112, 1124, 1133, 1150, 1152, 1162, 1181, 1198, 1202, 1204, 1220,
       1223, 1225, 1244, 1264, 1265, 1266, 1286, 1287, 1296, 1305, 1308,
       1312, 1357, 1374, 1380, 1383, 1392, 1398, 1404, 1406, 1409, 1413,
       1423, 1425, 1426, 1439, 1461, 1475, 1510, 1511, 1514, 1517, 1535,
       1541, 1543, 1552, 1554, 1563, 1592, 1594, 1608, 1610, 1617, 1620,
       1632, 1636, 1639, 1641, 1642, 1658, 1660, 1667, 1678, 1682, 1709,
       1715, 1731, 1740, 1747, 1750, 1751, 1761, 17

## _Algorithm to track cells back in time that made it to landfall_

In [6]:
%%time
combined_cells_landfalls = []
for i, cell_ids in enumerate(np.unique(cells_landfall)): # looped through tracks found with AMF1 site
    if cell_ids in track['cell'].values: # checking if those cells(tracks) are within the original tracked dataframe
        # print(f'Cell {cell_ids} present: made it to landfall 😀')
        track_c = track[track['cell'] == cell_ids] # display it if it is within the tracked dataframe
        combined_cells_landfalls.append(track_c) # append the results you get a new dataFrame calld combined_tracks_AMF1
    else:
        print('Cells not present 🥲')
        
# Concatenate the dataframes(combined_tracks_AMD) along the rows axis with keys
combined_cells_landfall_present = pd.concat(combined_cells_landfalls)
combined_cells_landfall_present = combined_cells_landfall_present.reset_index(drop=True)
combined_cells_landfall_present['landfall'] = globe.is_land(combined_cells_landfall_present['latitude'], combined_cells_landfall_present['longitude'])

CPU times: user 177 ms, sys: 4.83 ms, total: 182 ms
Wall time: 532 ms


In [7]:
combined_cells_landfall_present.head()

frame  idx      hdim_1      hdim_2  num  threshold_value  feature  \
0      0    8  188.472610  361.877737   16               20        8   
1      1    7  192.015026  362.419857   11               20       61   
2      2    6  195.334382  362.598927   12               20      110   
3      3    6  200.713718  364.586398   14               20      151   
4      4    5  204.367833  365.550327   18               20      198   

                  time              timestr  projection_y_coordinate  \
0  2020-03-13 04:00:00  2020-03-13 04:00:00            -2.248473e+06   
1  2020-03-13 04:05:00  2020-03-13 04:05:00            -2.252015e+06   
2  2020-03-13 04:10:00  2020-03-13 04:10:00            -2.255334e+06   
3  2020-03-13 04:15:00  2020-03-13 04:15:00            -2.260714e+06   
4  2020-03-13 04:20:00  2020-03-13 04:20:00            -2.264368e+06   

   projection_x_coordinate   latitude  longitude  cell        time_cell  \
0            278877.737009  69.599108  17.070280     8  0 days 00:00:00   
1            279419.856805  69.566479  17.072855     8  0 days 00:05:00   
2            279598.927410  69.536276  17.067031     8  0 days 00:10:00   
3            281586.398199  69.485415  17.099990     8  0 days 00:15:00   
4            282550.326596  69.451293  17.112668     8  0 days 00:20:00   

    distance   mean dBZ    max dBZ  area($km^2$)  landfall  
0  57.716745  13.245867  34.567482     18.903717     False  
1  55.372297  12.974534  35.577388     14.922312     False  
2  53.078408  12.780750  34.063904      7.959446     False  
3  50.903743  12.708913  36.584485     15.919167     False  
4  49.448644  12.845599  38.092506     31.838018     False

In [8]:
len(combined_cells_landfall_present)

3383

## _track up to the time the cell first hit landfall (after it hits the first landfall, remove all the subsequent cells hitting landfall, we are not interested in it)_

In [9]:
%%time
cells_to_first_landfall = []
for i, cell_idx in enumerate(np.unique(cells_landfall)):
    df = combined_cells_landfall_present[combined_cells_landfall_present['cell'] == cell_idx]
    # # find the index of the first occurrence of True
    idx = df.loc[df['landfall']].index[0]
    # slice the DataFrame to keep only the rows up to and including that index
    df = df.loc[:idx]
    # set all subsequent occurrences of True to False
    df.loc[df['landfall'].shift(-1) == True, 'landfall'] = False
    # print(df)
    cells_to_first_landfall.append(df)
first_landfall = pd.concat(cells_to_first_landfall)

CPU times: user 510 ms, sys: 12 ms, total: 522 ms
Wall time: 550 ms


## _calculate relative time --- relative time = time - time_landfall_

In [10]:
%%time
cells_relative_time = []
for index, cell_id in enumerate(np.unique(first_landfall['cell'])):
    if cell_id:
        # print(cell_id)
        cells = first_landfall[first_landfall['cell'] == cell_id] # get all the cell_ids
    # calculate for the relative time ----- relative time = time - time_landfall
        all_cells_times = cells['timestr']; all_cells_times = pd.to_datetime(all_cells_times.values) # convert timestr to datetime
        landfall_time_each_cell = all_cells_times[::-1][0]
        # relative_time = np.absolute(all_cells_times - landfall_time_each_cell) # relative time
        relative_time = np.subtract(landfall_time_each_cell, all_cells_times)
        relative_time = relative_time.seconds // 60
        # print(cell_id, relative_time)
        cells_relative_time.append(relative_time)

CPU times: user 373 ms, sys: 995 µs, total: 374 ms
Wall time: 376 ms


In [11]:
cells_relative_time[:20]

[Int64Index([25, 20, 15, 10, 5, 0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([10, 5, 0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([40, 35, 30, 25, 20, 15, 10, 5, 0], dtype='int64'),
 Int64Index([5, 0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([5, 0], dtype='int64'),
 Int64Index([10, 5, 0], dtype='int64'),
 Int64Index([10, 5, 0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64')]

In [12]:
# cells_relative_time
cells_relative_times = pd.Series([item for sublist in cells_relative_time for item in sublist])
# first_landfall['relative_time'] = cells_relative_times
# cells_relative_times.values
first_landfall['relative_time'] = cells_relative_times.values

In [13]:
len(first_landfall)

1606

In [14]:
first_landfall.to_csv('../saved-files/threshold-20/all-cells-first_landfall-threshold-20.csv', index=False)

In [15]:
# final = first_landfall[['cell', 'latitude', 'longitude', 'distance', 'max dBZ','mean dBZ', 'area' ,'timestr', 'landfall']]
# final

# END 